In [27]:
FILE = "./Data35aWalkProdFaces.txt"

In [28]:
from itertools import takewhile
from collections import defaultdict
from toposort import toposort_flatten

In [29]:
def removewhile(pred, iterator):
    condition = True
    result = []
    for i in iterator:
        if not pred(i):
            condition = False
        if not condition:
            result.append(i)
    return result

x = [0,0,1,0,1]
assert removewhile(lambda x: x == 0, x) == [1, 0, 1]

In [30]:
lines = []
with open(FILE, "r") as f:
    for l in f.readlines():
        lines.append(l)
lines = lines[4:-6]

In [31]:
faces_desc = [[]]

for l in lines:
    if l == "\n":
        faces_desc.append([])
    else:
        faces_desc[-1].append(l.replace("\n", "").strip())

faces_desc.pop()

[]

In [32]:
final_list = []

def clean(f_descp):
    f_desc = [(x.strip(), y.strip()) for l in f_descp for x, y in [l.split("|")]]
    original_number = f_desc[0][0].split(" ")[0] 
    f_desc[0] = (f_desc[0][0].split(" ")[-1], f_desc[0][1])
    return (original_number, f_desc)

finals = {}
newnames = {}

for f_descp in faces_desc:
    orig, f_desc = clean(f_descp)
    
    faces = [[] for _ in range(len(list(x for x in f_desc[0][1].split(" ") if x)) + 1)]
    for f, s in f_desc:
        faces[0].append(f)
        for e, k in enumerate([x for x in s.split(" ") if x]):
            if k:
                faces[e+1].append(k)
            
    def transform(faces):
        return [list(removewhile(lambda x: x == 0, map(lambda x: int(x), [x for x in ff[:-1] if x != "--"])))
                for ff in faces if "".join(ff) not in ["", "--"]]
    faces = transform(faces)
    
    def goc(name):
        if not name:
            return None
        name = tuple(name)
        if name in newnames:
            return newnames[name]
        else:
            newnames[name] = len(newnames)
            return newnames[name]
    
    name = goc(faces[0])
    insf = goc(faces[1] if len(faces) > 1 else None)
    outf = [goc(f) for f in faces[2:]]
#     print(orig, faces[0], name, insf, outf, faces)
    finals[name] = (insf, outf)

dims = defaultdict(int)

for elem in toposort_flatten({k: set(v) for k, vv in finals.items() for _, v in [vv]}):
    dims[elem] = dims[finals[elem][0]] + 1

dims = {k: v - 1 for k, v in dims.items()}
del dims[None]

In [33]:
{k: (v, dims[k]) for k, v in finals.items()}

{0: ((None, []), 0),
 1: ((None, []), 0),
 2: ((None, []), 0),
 3: ((None, []), 0),
 4: ((None, []), 0),
 5: ((0, [4]), 1),
 6: ((0, [1]), 1),
 7: ((1, [4]), 1),
 8: ((1, [3]), 1),
 9: ((1, [2]), 1),
 10: ((2, [3]), 1),
 11: ((3, [4]), 1),
 12: ((5, [6, 9, 10, 11]), 2),
 13: ((5, [6, 9, 10, 11]), 2),
 14: ((5, [6, 9, 10, 11]), 2),
 15: ((5, [6, 8, 11]), 2),
 16: ((5, [6, 8, 11]), 2),
 17: ((5, [6, 7]), 2),
 18: ((7, [8, 11]), 2),
 19: ((8, [9, 10]), 2),
 20: ((12, [17, 18, 19]), 3),
 21: ((12, [14]), 3),
 22: ((12, [13]), 3),
 23: ((13, [14]), 3),
 24: ((14, [15, 19]), 3),
 25: ((15, [17, 18]), 3),
 26: ((15, [16]), 3),
 27: ((15, [16]), 3),
 28: ((16, [17, 18]), 3),
 29: ((20, [22, 23, 24, 27, 28]), 4),
 30: ((20, [21, 24, 25]), 4),
 31: ((21, [22, 23]), 4),
 32: ((25, [26, 28]), 4),
 33: ((26, [27]), 4),
 34: ((29, [30, 31, 32, 33]), 5),
 35: ((None, []), 0),
 36: ((None, []), 0),
 37: ((None, []), 0),
 38: ((None, []), 0),
 39: ((None, []), 0),
 40: ((35, [39]), 1),
 41: ((35, [36])

In [34]:
#
#
#
from collections import Counter
Counter(dims.values())

Counter({0: 10, 1: 26, 2: 86, 3: 257, 4: 557, 5: 737, 6: 368})